In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 19:49 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 19:49 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal *.csv  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/tbl0.csv': File exists
copyFromLocal: `/tmp/wordcount/tbl1.csv': File exists


In [5]:
%%hive

DROP TABLE IF EXISTS tbl0;
DROP TABLE IF EXISTS tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 15.117 seconds
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.148 seconds


In [6]:
%%hive

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.954 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.887 seconds


In [7]:
%%hive

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.068 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.24 seconds


In [47]:
%%hive
DROP TABLE IF EXISTS let_by_year;
CREATE TABLE let_by_year AS
SELECT a.year_s, a.let, COUNT(*)
FROM(
SELECT YEAR(c4) AS year_s, lets AS let 
FROM tbl0 
LATERAL VIEW explode(c5) tbl0 AS lets) a
GROUP BY a.year_s, a.let
ORDER BY a.year_s ASC, a.let ASC;

DROP TABLE IF EXISTS let_by_year;
OK
Time taken: 0.015 seconds
CREATE TABLE let_by_year AS
SELECT a.year_s, a.let, COUNT(*)
FROM(
SELECT YEAR(c4) AS year_s, lets AS let 
FROM tbl0 
LATERAL VIEW explode(c5) tbl0 AS lets) a
GROUP BY a.year_s, a.let
ORDER BY a.year_s ASC, a.let ASC;
Query ID = root_20200218203145_172ea40e-51e5-4499-bfc9-9d9bd6bdeead
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0013, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0013/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0013
Hadoop job information for Stage-1: number of mappers: 1; 

In [48]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM let_by_year;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM let_by_year;
Query ID = root_20200218203244_717ae767-979e-4fd0-9185-ef9e06eda2e6
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0015, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0015/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0015
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 20:32:52,898 Stage-1 map = 0%,  reduce = 0%
2020-02-18 20:32:59,899 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.94 sec
MapReduce Total cumulative CPU time: 1 seconds 940 msec
Ended Job = job_1582053079980_0015
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_

In [49]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [50]:
!hdfs dfs -cat /tmp/output/000000_0 | head

2014,a,1
2014,b,1
2014,d,2
2014,e,1
2015,a,3
2015,b,1
2015,c,3
2015,d,2
2015,e,2
2016,a,2


In [51]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [52]:
!cat output/000000_0

2014,a,1
2014,b,1
2014,d,2
2014,e,1
2015,a,3
2015,b,1
2015,c,3
2015,d,2
2015,e,2
2016,a,2
2016,b,1
2016,c,3
2016,d,3
2016,e,3
2017,a,1
2017,b,1
2017,c,1
2017,e,1
2018,a,1
2018,d,1
